In [ ]:
# default_exp preprocessing.data

In [ ]:
#all_slow

# Data

> Classes and functions to load dataset, clean and save for further processing & vocab creation.

In [ ]:
import pandas as pd
import os

## Analyze Data
- Load all 3 datasets
- Merge into single df?
- Look for class imbalance
- Clean
- Split

In [ ]:
DATASTORE = '/Users/Vinod/code/datasets/peptide'
os.listdir(DATASTORE)

['.DS_Store', 'dna_binding', 'amp', 'acp']

In [ ]:
os.listdir(f'{DATASTORE}')

In [ ]:
amp_df = pd.read_csv(f'{DATASTORE}/amp/')